# Panel 3

### `import packages.needed`

In [17]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
#
from lrgsglib.core import *
#
move_to_rootf(print_tf=True)
plt.style.use('ipynb/nb_plotsheet.mplstyle')
#
path_paper = PATHPLOT / Path(PATHNPAPR)
path_panel3 = path_paper / Path('panel_3')
path_eris_xerr = path_panel3 / Path('er_ising_xerr')
path_phase_diagram = path_panel3 / Path('phase_diagram')
path_l2dcorrect = path_panel3 / Path('l2d_squared_cluster')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current working directory: /home/opisthofulax/PhD/projects/complex-networks/lrgsglib


### Fig. (a): Phase Diagram 
Expressions for the Nishimori line as function of $T$ and $p$

$$\mathrm{nishi}_p(T)=\frac12​(1+e^{k_B ​TJ}​)-1,\qquad
\mathrm{nishi}_T(p) = \frac{J}{k_B \ln\left(\frac{1}{2p} - 1\right)}$$

In [9]:
phd = 'serr' or 'xerr'
J, k_B, p_c = 1, 1, 0.103
nishi_p = lambda T: 0.5 * (1 + np.exp(J / (k_B * T)))**(-1)
nishi_T = lambda p: J * (k_B*np.log(1/(2*p)-1))**(-1)
#
T_c, T_max, p_max = 2.2, 2.5, 0.5
TlinSpace = np.linspace(1e-2, T_max, 100)
#
kwNishiLine = dict(ls='-.', lw=1, marker='', c='k', clip_on=True)
kwCritPoints = dict(marker='o', c='r', clip_on=False, zorder=1e3)
kwSgLine = dict(lw=2, ls='-', c='#8030ff', marker='', clip_on=False, zorder=9e1)

In [10]:
fig, ax = plt.subplots(figsize=(8, 5))
#
ax.set_ylim(0, T_max)
ax.set_xlim(0, p_max)
#
match phd:
    case 'xerr':
        xticks = [0, p_c, 0.4, 0.5]
        xticklabels = [r"0", r"\(p_{\mathrm{c}}^{(1)}\)", r"\(p_{\mathrm{c}}^{(2)}\)", r"0.5"]
        p_c1 = 0.4
    case 'serr':
        p_c1 = p_c
        xticks = [0, p_c, 0.5]
        xticklabels = [r"0", r"\(p_{\mathrm{c}}\)", r"0.5"]
        ax.text(p_max/3, T_max/2, r'\emph{Nishimori Line}', fontsize=16)
        ax.plot(nishi_p(TlinSpace), TlinSpace, **kwNishiLine)
#
critPoints = np.array([[0, p_c1], [T_c, 0]])
pSglinSpace = np.linspace(p_c1, p_max)
TSglinSpace = 0*np.ones(len(pSglinSpace))
yticks = [0, T_c]
yticklabels = [r"0", r"\(T_{\mathrm{c}}^{\text{\Large Ising}}\)"]
#
ax.text(p_c1-.05*p_c1, 0.1, r'\emph{Spin Glass}', c='#8030ff', fontsize=20)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_xticklabels(xticklabels)
ax.set_yticklabels(yticklabels)
#
ax.scatter(*critPoints, **kwCritPoints)
ax.plot(pSglinSpace, TSglinSpace, **kwSgLine)
#
ax.tick_params(axis='both', which='both', labelsize=30)
#
#
fig.savefig(os.path.join(path_phase_diagram, f'phase_diagram_{phd}.pdf'), 
            bbox_inches='tight', transparent=True)
plt.close('all')

### Fig. (e): Phase diagram numercal 

In [18]:
side = 64
neigen = 2
cell = 'randXERR'
tt = 'long'

In [19]:
lattice = Lattice2D(side, path_data=path_l2dcorrect)
fnames = os.listdir(lattice.isingpath)
p_values = extract_values_from_filenames(fnames, r"p=([\d.]+)", unique=True)
p_T_dict = {p: extract_values_from_filenames(fnames, rf"p={p}_T=([\d.]+)") 
            for p in p_values}

In [21]:
averages = []
for p in p_values:
    for T in p_T_dict[p]:
        filename = f"outcl0_p={p}_T={T}_gs{neigen}{cell}{tt}.txt"
        filepath = os.path.join(lattice.isingpath, filename)
        if os.path.exists(filepath):
            try:
                data = np.loadtxt(filepath)
                data = data[np.all(np.abs(data) <= 1, axis=1)]
                avg_abs_col1 = np.mean(np.abs(data[:, 0]))
                averages.append((float(p), float(T), avg_abs_col1))
            except:
                pass
averages = np.array(averages)

In [22]:
p_values_plot = averages[:, 0]
T_values_plot = averages[:, 1]
avg_abs_col1_plot = averages[:, 2]
x, y, z = interpolate_grid_data(p_values_plot, T_values_plot, avg_abs_col1_plot)
z_smoothed = gaussian_filter(z, sigma=3)  # Adjust sigma for more/less smoothing

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
cnt = ax.contourf(x, y, z_smoothed, levels=50, cmap=restr_twilight)
div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')
cb = fig.colorbar(cnt, cax=cax)

ax.set_xlabel(r'$p$', labelpad=20)
ax.set_ylabel(r'$T$', labelpad=20)
cb.set_label(r'$\expval{m_{\infty}^{\mathrm{GC}}}$', loc='top')


for c in cnt.collections:
    c.set_edgecolor("face")


fig.savefig(path_l2dcorrect / Path(f'PhDiaL2D_{side}_{cell}.pdf'), 
                bbox_inches='tight', transparent=True)
plt.close('all')